# Generative AI Use Case: Summarize Dialogue

Kita akan mengambil kumpulan data dari percakapan yang terjadi antara orang-orang. Rencana kita adalah untuk merangkum dialog-dialog ini, jadi bayangkan dialog dukungan antara Anda dan pelanggan Anda, mungkin di akhir bulan Anda ingin merangkum semua masalah yang telah ditangani tim dukungan pelanggan Anda pada bulan itu.

In [ ]:
%pip install transformers
%pip install datasets
    #ransformers \
    #atasets  --quiet

Ini akan mengimpor fungsi yang disebut load data-set, ini akan mengimpor
beberapa model dan tokenizer yang diperlukan untuk menyelesaikan lab kita di sini.

## 1. Set up Kernel and Required Dependencies

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

Kami akan menggunakan data-set yang disebut Dialogue sum dan ini adalah data-set publik yang diunggulkan oleh transformers, dan khususnya pustaka data-set memberi kami akses ke sana, jadi yang kami lakukan adalah memanggil load data-set yang diimpor di atas dan kami memuat data-set ini.

## 2. Summarize Dialogue without Prompt Engineering

In [ ]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Sekarang, dari sini ke depan, kami akan menjelajahi beberapa data, kami akan mencoba **merangkum dengan hanya menggunakan model dasar flat T5.**

In [ ]:
example_indices =[40,200]
dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example', i+1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Exam

         

*   Berikut adalah contoh percakapan antara orang 1 dan orang 2.
*   Orang 1 berkata, "jam berapa ini, Tom?" Tampaknya nama orang 2 adalah Tom sebenarnya.
*   Cukup sebentar, jam 9 kurang 10 menurut jam saya dan seterusnya.
*   **Berikut adalah ringkasan dasar manusia.**
*   **Ini adalah apa yang seorang manusia telah tandai sebagai ringkasan percakapan itu.**
*   Sekarang kita akan mencoba meningkatkan ringkasan itu dengan menggunakan model kita.
*   Sekali lagi, **tidak ada model yang dimuat.**
*   Ini benar-benar hanya data aktual.
*   Inilah percakapannya dan kemudian pikirkan ini seperti ini adalah sampel pelatihan dan kemudian ini adalah apa yang manusia tandai dan kemudian kita akan membandingkan ringkasan manusia, yang kita anggap sebagai dasar, kita akan membandingkannya dengan apa yang diprediksi oleh model sebagai ringkasan.
*   Model akan benar-benar menghasilkan ringkasan.
















*   Berikut adalah contoh kedua.
*   Anda dapat melihat ada beberapa istilah yang familiar di sini yang banyak dari kita kenal, CD ROM program melukis untuk perangkat lunak Anda.



Sekarang, di sini kita akan **memuat model FLAN-T5.**

Ini adalah model tujuan umum yang sangat bagus yang dapat melakukan banyak tugas dan hari ini kita akan fokus pada kemampuan FLAN-T5, untuk merangkum percakapan.

In [ ]:
model_name='google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

     
*   **Setelah memuat model, kita harus memuat tokenizer**.
*   Sekarang, semua ini berasal dari pustaka Hugging Face Transformers.
*   Untuk memberi Anda contoh, sebelum transformers hadir, kami harus menulis banyak kode ini sendiri.
*   Bergantung pada jenis model, sekarang ada banyak model bahasa yang berbeda dan beberapa dari mereka melakukan hal-hal dengan cara yang sangat berbeda dari beberapa model lainnya.
*   Ada banyak pustaka ad hoc yang disesuaikan yang mencoba melakukan hal-hal yang serupa.
*   Kemudian Hugging Face datang dan benar-benar memiliki implementasi yang sangat dioptimalkan dari semua ini.







In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

*   Sekarang, di sini ada tokenizer.
*   Ini adalah yang akan digunakan untuk mengubah
teks mentah dari percakapan kita ke dalam ruang vektor kita yang kemudian dapat diproses oleh model Flan-T5 kita.



Hanya untuk memberi Anda gambaran, mari kita ambil contoh kalimat di sini.

In [ ]:
sentence = "What time is it, Tom?"

sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(
        sentence_encoded["input_ids"][0],
        skip_special_tokens=True)

print('Encoded Sentence:')
print(sentence_encoded["input_ids"][0])
print('Decoded Sentence:')
print(sentence_decoded)

Encoded Sentence:
tensor([ 363,   97,   19,   34,    6, 3059,   58,    1])
Decoded Sentence:
What time is it, Tom?


*   "Jam berapa ini, Tom?" Kalimat pertama dari percakapan kita di atas, kita melihat kalimat yang dienkripsi sebenarnya adalah angka-angka di sini.
*   Kemudian jika Anda mendekode-nya, kita lihat bahwa ini mendekode kembali ke aslinya.
*   **Tugas tokenizer adalah mengonversi teks mentah menjadi angka.**
*   Angka-angka itu menunjuk ke serangkaian vektor atau embedding seperti yang sering disebut, yang kemudian digunakan dalam operasi matematika seperti pembelajaran mendalam, propagasi balik, aljabar linear kita, semua itu hal yang menyenangkan.





**Sekarang, setelah kami memuat model kami dan kami memuat tokenizer kami**, kami dapat **menjalankan beberapa percakapan ini melalui model Flan-T5** dan melihat apa yang sebenarnya dihasilkan oleh model ini sebagai ringkasan untuk percakapan ini.

In [ ]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    inputs = tokenizer(dialogue, return_tensors='pt')
    outputs = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True)

    print(dash_line)
    print('Example', i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{outputs}\n')

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - WITHOUT PROMPT ENGINEERING:
Person1: It's ten to nine.

--------------------------------

*   Di sini lagi, kami memiliki percakapan.
*   Di sini lagi adalah ringkasan dasar.
*   **Kemudian kita melihat tanpa ada prompt engineering sama sekali, hanya mengambil percakapan aktual, melewatkannya ke model Flan-T5 kami, itu tidak melakukan pekerjaan yang sangat baik dalam merangkum.**
*   Kami melihat pukul sembilan kurang 10.
*   Itu tidak sangat membantu.
*   Ada beberapa detail lain dalam percakapan ini yang saat ini tidak terungkap dalam ringkasan.
*   Sama dengan percakapan tentang CD-ROM kita, ringkasan dasarnya adalah Person 1 mengajarkan Person 2 cara meningkatkan perangkat lunak dan perangkat keras di sistem Person 2.
*   Model menghasilkan Person 1 sedang memikirkan tentang meningkatkan komputernya.
*   Sekali lagi, **banyak detail dalam percakapan asli ini yang tidak tercermin dalam ringkasan.**
*    Mari kita lihat bagaimana kita bisa meningkatkan ini.

## 3. Summarize Dialogue with an Instruction Prompt

### Zero-Shot Inference

Dalam pelajaran, **Anda belajar cara menggunakan instruksi untuk memberi tahu model Anda apa yang Anda coba lakukan dengan data yang Anda berikan padanya. Berikut adalah contoh**. Ini disebut pembelajaran konteks dan khususnya *zero-shot inference* dengan instruksi. Berikut adalah instruksi, yang merangkum percakapan berikutnya. **Inilah percakapan aktual, dan kemudian kita memberi tahu model di mana harus mencetak ringkasannya, yang akan berada setelah kata ringkasan ini.** Sekarang ini terlihat sangat sederhana dan mari kita lihat apakah ini berhasil. Mari kita lihat apakah hal-hal menjadi lebih baik. Tidak banyak yang lebih baik di sini.



In [ ]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    # prompt
    prompt = f"""
    Summarize the following conversation
    {dialogue}
    Summary:
    """



    inputs = tokenizer(prompt, return_tensors='pt') # prompt
    outputs = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True)

    print(dash_line)
    print('Example', i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{outputs}\n')

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

    Summarize the following conversation
    #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
    Summary:
    
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
The train is 

*   Baseline masih Person 1 sedang terburu-buru, Tom memberitahu Person 2 bahwa masih banyak waktu.
*   Kemudian pembelajaran konteks *zero-shot* dengan prompt, hanya mengatakan kereta akan berangkat.
*   Sekali lagi, **bukan yang terbaik.**
*   Dan kemudian inilah *zero-shot* untuk contoh komputer.
*   Masih berpikir bahwa Person 1 mencoba meningkatkan, jadi **tidak banyak yang lebih baik.**







Ada prompt yang berbeda yang dapat kita gunakan di sini, yaitu di mana kita hanya mengatakan ***dialogue corn***. Sekarang ini benar-benar terserah Anda. **Ini adalah sisi prompt engineering dari LLM ini di mana kita mencoba menemukan prompt terbaik dan dalam hal ini hanya *zero-shot inference*. Tidak ada *fine-tuning* model, tidak ada apa-apa.** Yang kita lakukan hanyalah menemukan instruksi yang berbeda untuk dilewatkan dan melihat apakah model melakukannya lebih baik dengan frase yang sedikit berbeda. Mari kita lihat bagaimana ini.

In [ ]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    # prompt engineering
    prompt = f"""
Dialogue:

{dialogue}

What was going on?
"""

    inputs = tokenizer(prompt, return_tensors='pt')
    outputs = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True)

    print(dash_line)
    print('Example', i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{outputs}\n')

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
    
What was going on?

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Tom is late for the train.

------------

Ini benar-benar merupakan kebalikan dari sebelumnya di mana **di sini kita hanya mengatakan inilah percakapannya, dan kemudian setelahnya kita mengatakan apa yang terjadi di dalam percakapan itu**. Mari kita lihat apakah ini melakukan sesuatu yang lebih baik. Tom terlambat untuk kereta, jadi itu diambilnya, **tetapi masih tidak terlalu bagus**. Di sini kita lihat Person 1. Anda bisa menambahkan program lukisan. Person 2 itu akan menjadi bonus. Sedikit lebih baik. **Itu tidak benar-benar tepat, tetapi semakin baik.** Setidaknya mengambil beberapa nuansa.

Sekarang, sebagai bagian dari pembelajaran konteks, Anda belajar ada yang disebut ***one-shot inference dan few shot inference***. Mari kita dapatkan contohnya di sini. Mari kita berlatih langsung dengan *one-shot* dan kemudian *few shot*. Sebelumnya kita melakukan ***zero-shot***. Itu berarti kita **tidak memberinya sampel dari prompt dan kemudian penyelesaian, semua yang kita lakukan hanya memberinya prompt**. Kami meminta model untuk melakukan sesuatu dan melihat apa yang dihasilkan model. Dengan ***one-shot*** dan kemudian ***few shot***, kita akan m**emberinya sampel yang benar, atau yang menggunakan dasar manusia.** Itu memberi model sedikit lebih banyak informasi untuk bekerja.

## 4. Summarize Dialogue with One Shot and Few Shot Inference

### One-Shot Inference

Mari kita lihat bagaimana ***one-shot inference*** bekerja di sini. Yang kita lakukan hanyalah **mengambil contoh lengkap, termasuk ringkasan dari dasar manusia, kemudian memberinya contoh kedua, tetapi tanpa ringkasan sebenarnya. Itu adalah percakapan yang ingin kita ringkas model.** Mari kita lihat bagaimana ini terlihat.

In [ ]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt =''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']

        prompt +=f"""

# yang kita lakukan hanyalah mengambil contoh lengkap, termasuk ringkasan dari dasar manusia

Dialogue:
{dialogue}
What was going on?
{summary}
"""
#kemudian memberinya contoh kedua, tetapi tanpa ringkasan sebenarnya.

    dialogue = dataset['test'][example_index_to_summarize]['dialogue']

    prompt +=f"""
Dialogue:
{dialogue}
What was going on?
"""
    return prompt


***One-shot*** berarti saya **memberinya satu contoh lengkap, termasuk jawaban yang benar seperti yang ditetapkan oleh manusia di sini, dasar manusia. Kemudian kami memberinya contoh kedua dan bertanya pada model apa yang terjadi.**

In [ ]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)


Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

Dialogue:
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a 

Mari kita lihat bagaimana kita melakukannya di sini.

In [ ]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors ='pt')
outputs = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens = 50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'INPUT PROMPT:\n{one_shot_prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOTS:\n{outputs}')


---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

Dialogue:
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Pe

Di sini kami hanya akan melakukan peningkatan perangkat lunak. Person1 ingin meningkatkan, Person2 ingin menambahkan program lukisan, Person1 ingin menambahkan CD ROM. Saya pikir **sedikit lebih baik** dan mari kita lanjutkan.

### Few shot inference

Ada yang disebut ***few shot inference*** juga. Sekarang beberapa dari Anda mungkin bertanya, nah, ini tampaknya seperti kecurangan karena kami sebenarnya memberinya satu jawaban dan kemudian menanyakan padanya. Ini tidak benar-benar curang. **Ini lebih seperti Anda membantu model membantu dirinya sendiri**. Sekarang dalam pelajaran dan lab yang akan datang, kita akan *fine-tuning* model di mana kita bisa kembali ke *zero-shot inferences*, yang merupakan apa yang biasanya Anda pikirkan sebagai model bahasa yang baik. Tetapi di sini kita hanya membangun beberapa intuisi. Ingatlah, ini adalah cara yang sangat murah untuk mencoba model ini dan bahkan untuk mencari tahu model mana yang seharusnya Anda *fine-tune*. Kami memilih **Plan T5 karena bekerja di sejumlah besar tugas.**  Tetapi jika Anda tidak memiliki ide bagaimana modelnya, jika Anda hanya mendapatkannya dari suatu tempat di model hub. Ini adalah langkah pertama. ***Prompt engineering, zero-shot, one-shot, few shot* hampir selalu langkah pertama ketika Anda mencoba mempelajari model bahasa yang Anda terima dan kumpulan data.** Juga sangat spesifik terhadap kumpulan data serta tugasnya.

In [ ]:
example_indices_full = [40, 80, 120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

Dialogue:
#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and picnic blanket.
#Person1#: All set. May, 

***Few shot*** berarti bahwa **kita memberikan tiga contoh lengkap, termasuk ringkasan dasar manusia, 1, 2, 3, dan kemudian yang keempat tetapi tanpa ringkasan manusia.** Ya, meskipun kita memilikinya, kita hanya menjelajahi model kami saat ini. Kami mengatakan, beritahu kami apa itu dialog keempat itu. Itu ringkasan. Hanya abaikan beberapa kesalahan ini. Beberapa urutan ini sedikit lebih besar dari panjang konteks 512 model. Biasanya, Anda mungkin ingin menyaring semua masukan yang lebih besar dari 512. Tetapi di sini masih melakukan pekerjaan yang cukup baik.

In [ ]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors ='pt')
outputs = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens = 50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOTS:\n{outputs}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOTS:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.


Di sini kita melihat kasus di mana ***few shot* tidak banyak lebih baik dari *one-shot*.** Ini adalah sesuatu yang perlu Anda perhatikan karena dalam praktiknya, orang sering mencoba hanya terus menambahkan lebih banyak tembakan, *five-shot*, *six-shot*. Biasanya, berdasarkan pengalaman saya, **di atas lima atau enam tembakan, jadi prompt lengkap dan kemudian penyelesaian, Anda benar-benar tidak mendapatkan banyak setelah itu**. Entah model bisa melakukannya atau tidak dan berlangsung sekitar lima atau enam. Di sini kita melihat **untuk sampel tertentu ini bahwa sebenarnya *one-shot* sudah cukup baik.**

## 5. Generative Configuration Parameters for Inference

Sekarang bagian terakhir dari lab ini akan menyenangkan. Ini adalah tempat di mana Anda benar-benar dapat bermain dengan beberapa parameter konfigurasi ini yang Anda pelajari selama pelajaran. Hal-hal seperti sampling, temperatur. Anda bisa bermain dengan ini mencoba dan mendapatkan intuisi Anda tentang bagaimana hal-hal ini dapat memengaruhi apa yang sebenarnya dihasilkan oleh model. Dalam beberapa kasus, misalnya, dengan meningkatkan temperatur ke atas, menuju satu atau bahkan lebih dekat ke dua, Anda akan mendapatkan tanggapan yang sangat kreatif. Jika Anda menurunkannya saya percaya 0.1 adalah minimum untuk implementasi hugging face apa pun, dari kelas konfigurasi generasi ini yang digunakan ketika Anda benar-benar menghasilkan. Saya bisa melewati konfigurasi generasi langsung di sini. **Jika Anda turun ke 0.1, itu akan membuat respons lebih konservatif dan seringkali akan memberikan respons yang sama berulang kali.** **Jika Anda lebih tinggi, saya percaya sebenarnya 2.0 adalah yang tertinggi.** Jika Anda mencoba 2.0, itu akan mulai memberikan beberapa tanggapan yang sangat liar. Ini menyenangkan. Anda harus mencoba.

In [ ]:
generation_config = GenerationConfig(max_new_tokens=50)
# generation_config = GenerationConfig(max_new_tokens=10)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.1)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=1.0)

inputs = tokenizer(few_shot_prompt, return_tensors ='pt')
outputs = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config=generation_config,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOTS:\n{outputs}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOTS:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.
